In [1]:
import os
import numpy as np

import torch
import torch.nn as nn

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
img_size = 32 # size of each image dimension
channels = 3 # number of image channels
img_shape = (channels, img_size, img_size)
hidden_dim = 64
lr = 0.0002
n_cpu = os.cpu_count()//2 
batch_size = 256 if torch.cuda.is_available() else 64
n_epochs = 10 
noise_dim = 100 # dimensionality of the latent space

In [3]:
data_path = '../data'
os.makedirs(data_path, exist_ok=True)

In [4]:
# Configure data loader
transform = transforms.Compose([transforms.Resize(img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])])

loader_kwargs = {'num_workers': n_cpu, 'pin_memory': True} 

train_data = datasets.CIFAR10(root=data_path, train=True, download=True, transform=transform)
test_data = datasets.CIFAR10(root=data_path, train=False, download=True, transform=transform)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=False, **loader_kwargs)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, **loader_kwargs)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
class Generator(nn.Module):
    def __init__(self, img_size=img_size):
        super(Generator, self).__init__()
        
        self.lin = nn.Linear(noise_dim, hidden_dim * img_size * img_size)

        self.conv = nn.Sequential(
            nn.BatchNorm2d(hidden_dim),
            nn.Conv2d(hidden_dim, hidden_dim, 3, stride=1, padding=1),
            nn.BatchNorm2d(hidden_dim),
            nn.LeakyReLU(True),
            nn.Conv2d(hidden_dim, hidden_dim, 3, stride=1, padding=1),
            nn.BatchNorm2d(hidden_dim),
            nn.LeakyReLU(True),
            nn.Conv2d(hidden_dim, channels, 3, stride=1, padding=1),
            nn.Tanh(),
        )

    def forward(self, z):
        out = self.lin(z)
        out = out.view(out.shape[0], hidden_dim, img_size, img_size)
        img = self.conv(out)
        return img

In [6]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Conv2d(channels, hidden_dim, 3,padding=1),
            nn.LeakyReLU(True),
            nn.Dropout2d(),
            nn.Conv2d(hidden_dim, hidden_dim, 3,padding=1),
            nn.LeakyReLU(True),
            nn.Dropout2d(),
            nn.BatchNorm2d(hidden_dim),
            nn.Conv2d(hidden_dim, hidden_dim, 3,padding=1),
            nn.LeakyReLU(True),
            nn.Dropout2d(),
            nn.BatchNorm2d(hidden_dim),
        )

        self.adv_layer = nn.Sequential(
            nn.Linear(hidden_dim * img_size * img_size, 1), 
            nn.Sigmoid()
        )

    def forward(self, img):
        out = self.model(img)
        out = out.view(out.shape[0], -1)
        validity = self.adv_layer(out)

        return validity

In [7]:
criterion = torch.nn.BCELoss()

generator = Generator()
discriminator = Discriminator()

optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr)
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr)

generator.to(device)
discriminator.to(device)

Discriminator(
  (model): Sequential(
    (0): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): LeakyReLU(negative_slope=True)
    (2): Dropout2d(p=0.5, inplace=False)
    (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): LeakyReLU(negative_slope=True)
    (5): Dropout2d(p=0.5, inplace=False)
    (6): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): LeakyReLU(negative_slope=True)
    (9): Dropout2d(p=0.5, inplace=False)
    (10): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (adv_layer): Sequential(
    (0): Linear(in_features=131072, out_features=1, bias=True)
    (1): Sigmoid()
  )
)

In [8]:
saved_dir = 'dcgan_images'
os.makedirs(saved_dir, exist_ok = True)

In [9]:
for epoch in range(n_epochs):
    for i, (imgs, _) in enumerate(train_loader):

        # ground truths
        real = torch.ones(imgs.size(0), 1, requires_grad=False, device=device)
        fake = torch.zeros(imgs.size(0), 1, requires_grad=False, device=device)

        real_imgs = imgs.to(device)

        # ===== Generator =====
        optimizer_G.zero_grad()

        # Sample noise
        z = torch.normal(0, 1, (imgs.shape[0], noise_dim), device=device)

        gen_imgs = generator(z)

        loss_G = criterion(discriminator(gen_imgs), real)
        loss_G.backward()
        optimizer_G.step()

        # ===== Discriminator =====
        optimizer_D.zero_grad()

        real_loss = criterion(discriminator(real_imgs), real)
        fake_loss = criterion(discriminator(gen_imgs.detach()), fake)
        loss_D = (real_loss + fake_loss) / 2

        loss_D.backward()
        optimizer_D.step()
        
    # ===== save images and print logs =====
    save_image(gen_imgs.data[:25], f"{saved_dir}/{epoch+1}.png", nrow=5, normalize=True)
    print(f"epoch: {epoch+1}/{n_epochs}, D loss: {loss_D.item():.4f}, G loss: {loss_G.item():.4f}")

epoch: 1/10, D loss: 1.2911, G loss: 1.0306
epoch: 2/10, D loss: 1.0228, G loss: 1.0456
epoch: 3/10, D loss: 0.7685, G loss: 0.8461
epoch: 4/10, D loss: 0.8162, G loss: 0.9701
epoch: 5/10, D loss: 0.6142, G loss: 0.5585
epoch: 6/10, D loss: 0.7343, G loss: 0.9574
epoch: 7/10, D loss: 0.6266, G loss: 0.8637
epoch: 8/10, D loss: 0.6863, G loss: 0.6343
epoch: 9/10, D loss: 0.7004, G loss: 0.5729
epoch: 10/10, D loss: 0.7632, G loss: 0.7548
